# Tópico 3 – Validação cruzada + Análise de Erros

Este notebook aplica validação cruzada e análise de erros utilizando dados da API da Câmara dos Deputados para prever se proposições serão aprovadas.

In [ ]:

import requests
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:

# 1. Baixar votações
url = "https://dadosabertos.camara.leg.br/api/v2/votacoes"
params = {"itens": 100}
res = requests.get(url, params=params)
dados = res.json()['dados']


In [ ]:

# 2. Criar dataframe
df = pd.DataFrame(dados)
df['aprovado'] = df['resultado'].apply(lambda x: 1 if isinstance(x, str) and 'Aprovado' in x else 0)


In [ ]:

# 3. Pré-processamento
df['dataHoraRegistro'] = pd.to_datetime(df['dataHoraRegistro'])
df['ano'] = df['dataHoraRegistro'].dt.year
df['mes'] = df['dataHoraRegistro'].dt.month
df.fillna('desconhecido', inplace=True)

df_encoded = pd.get_dummies(df[['tipoVotacao', 'descricaoTipo']], drop_first=True)
X = pd.concat([df[['ano', 'mes']], df_encoded], axis=1)
y = df['aprovado']


In [ ]:

# 4. Treinar modelo com validação cruzada
model = RandomForestClassifier(random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=cv, scoring='f1_macro')
scores


In [ ]:

# 5. Treinar em treino/teste e analisar
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [ ]:

# 6. Gráfico dos scores
sns.boxplot(data=[scores])
plt.title("Distribuição dos F1-macro nas folds")
plt.ylabel("F1-score")
plt.show()


In [ ]:

# 7. Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Matriz de Confusão")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.show()
